<a href="https://colab.research.google.com/github/kmouts/PPS_MultiComms/blob/master/video_analytics_with_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>